In [ ]:
import logging

# logging.basicConfig(level=logging.INFO)
logging.basicConfig(level=logging.WARN)

from data import Data
from engine import Engine, TreeClimber
from html_builder import HTMLBuilder
from krakow import krakow
from krakow.utils import create_dendrogram, normalized_dasgupta_cost

import matplotlib.pyplot as plt

import io
from PIL import Image

import numpy as np

In [ ]:
def _depth_of_leaves(tree):
    if tree.is_leaf():
        return [0]
    
    left = _depth_of_leaves(tree.left)
    right = _depth_of_leaves(tree.right)

    return [depth + 1 for depth in left + right]

def get_disbalance(tree):
    """
    how much higher is the average leaf depth, than in an ideally balanced tree
    """
    depths = _depth_of_leaves(tree)
    return np.average(depths) - np.log2(tree.count)


def _balance_on_single_node(node):
    """
    ideal balance (0.5, 0.5) gives a score 0
    any inbalance returns negative score
    """
    if node.is_leaf() or node.left.is_leaf() or node.right.is_leaf():
        # TODO is this robust? maybe give some penalty instead
        # this will never be a problem if we stay near the top layers
        return 0
    left_ratio = node.left.count / node.count
    right_ratio = node.right.count / node.count
    return 2 + np.log2(left_ratio) + np.log2(right_ratio)


def get_top_levels_balance_log(tree, levels=5):
    """
    how disbalanced are top levels of the tree
    any inbalance returns negative score, 0 is perfect
    """
    balance_on_levels = []

    nodes = [tree]
    for _ in range(levels):
        balances_on_one_level = [_balance_on_single_node(node) for node in nodes]
        balance_on_levels.append(np.average(balances_on_one_level))

        new_nodes = []
        for node in nodes:
            if node.is_leaf():
                continue
            if node.left is not None:
                new_nodes.append(node.left)
            if node.right is not None:
                new_nodes.append(node.right)
        nodes = new_nodes

    return np.average(balance_on_levels)

In [ ]:
for alpha in [1, 1.5, 2, 2.5, 3, 3.5, 4]:
    data = Data(alpha=alpha, use_cached_forum_data=True)
    img = create_dendrogram(data._dendrogram)
    print(f"\nalpha = {alpha}")
    print(f"Disbalance: {get_disbalance(data.tree):.3f}")
    print(f"Top levels balance: {get_top_levels_balance_log(data.tree):.3f}")
    print(f"Clustering quality: {1 - normalized_dasgupta_cost(data.Tag_cooccurence, data._dendrogram):.3f}")
    display(Image.open(img))

In [ ]:
balances = []
top_balances = []
clustering_qualities = []
alphas = np.linspace(1, 3, num=21)
for alpha in alphas:
    data = Data(alpha=alpha, use_cached_forum_data=True)
    balances.append(get_disbalance(data.tree))
    top_balances.append(get_top_levels_balance_log(data.tree, levels=5))
    clustering_qualities.append(1 - normalized_dasgupta_cost(data.Tag_cooccurence, data._dendrogram))

In [ ]:
# plot balances and clusterig qualities, with different scales
plt.plot(alphas, top_balances, label="top levels balance")
plt.legend()

In [ ]:
plt.plot(alphas, clustering_qualities, label="clustering quality")
plt.legend()

In [ ]:
plt.plot(alphas, balances, label="disbalance")
plt.legend()